In [1]:
import os
import csv
import json
import pickle
import requests
from time import sleep

Consumer Key	XZoHfEhHTquhRinLcNrS
Consumer Secret	QozVgMYFgZkYwcCdXNOhApHklUlVKUxe
Request Token URL	https://api.discogs.com/oauth/request_token
Authorize URL	https://www.discogs.com/oauth/authorize
Access Token URL	https://api.discogs.com/oauth/access_token

In [18]:
key = 'XZoHfEhHTquhRinLcNrS'
secret = 'QozVgMYFgZkYwcCdXNOhApHklUlVKUxe'

### Get release thumbs by artist (much faster)

In [28]:
releases = set([_[6:-4] for _ in os.listdir('thumbs')])

In [37]:
artist_id = 151642 #1

In [41]:
while True:
    print(len(releases), end='\r')
    while True:
        sleep(1)
        url = f'https://api.discogs.com/artists/{artist_id}/releases?key={key}&secret={secret}'
        r = requests.get(url, headers={'User-agent': 'FooBarApp/3.0'})
        if r.status_code == 200 or r.status_code == 404:
            break
    response = json.loads(r.text)
    if 'message' in response and response['message'] == 'Artist not found.':
        artist_id += 1
        continue

    for release in response.get('releases', []):
        image_url = release.get('thumb', '')

        if 'id' in release and image_url != '' and str(release['id']) not in releases:
            while True:
                r = requests.get(image_url, headers={'User-agent': 'FooBarApp/3.0'})
                if r.status_code == 200 or r.status_code == 404:
                    break
            if r.status_code == 200:
                with open(f'thumbs/thumb_{release["id"]}.jpg', 'wb') as file:
                    file.write(r.content)
            releases.add(str(release['id']))
                
    artist_id += 1

KeyboardInterrupt: 

In [40]:
artist_id

153329